#### 케라스에 포함된 IMDB 데이터는 미리 토큰화가 되어있기 때문에 원본 텍스트 데이터 부터 시작!!

### 원본 텍스트 데이터 내려받기
> http://mng.bz/0tIo 에서 원본 데이터셋을 내려받고 압축을 해제한다.

### IMDB 원본 데이터 전처리하기

In [7]:
import os

imbd_dir = './datasets/aclImdb'
train_dir = os.path.join(imbd_dir, 'train')

labels = []

texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)